In [1]:
import numpy as np
import types
import pandas as pd
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_4d38ea6703f6412e99fb999ad9c2d4dd = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='oU6vGo7dyTAmmt0flLMKoLNrm8Hsp1mfuKUEqOE4NWtr',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_4d38ea6703f6412e99fb999ad9c2d4dd.get_object(Bucket='fast-donotdelete-pr-npu2nnnjspus3r',Key='bikebuyer.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

ds= pd.read_csv(body)
ds.head()

#Encoding required *(er)
er=ds[['Marital_Status', 'Gender','Education','Occupation','Home_Owner','Region']].values
col=[['Marital_Status'], ['Gender'],['Education'],['Occupation'],['Home_Owner'],['Region']]
xc=ds[['Yearly_Income','Children','Cars','Commute_Distance','Age']]
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le=LabelEncoder()
#Creating a Label Encoded dataframe 
temp1=pd.DataFrame(le.fit_transform(er[:,0]),columns=col[0])
temp2=pd.DataFrame(le.fit_transform(er[:,1]),columns=col[1])
temp3=pd.DataFrame(le.fit_transform(er[:,2]),columns=col[2])
temp4=pd.DataFrame(le.fit_transform(er[:,3]),columns=col[3])
temp5=pd.DataFrame(le.fit_transform(er[:,4]),columns=col[4])
temp6=pd.DataFrame(le.fit_transform(er[:,5]),columns=col[5])

temp=pd.concat([temp1,temp2],axis=1)
temp=pd.concat([temp,temp3],axis=1)
temp=pd.concat([temp,temp4],axis=1)
temp=pd.concat([temp,temp5],axis=1)
temp=pd.concat([temp,temp6],axis=1)
#------------------------------------------------------------------------------
x=pd.concat([xc,temp],axis=1)
x=x.dropna()
h=pd.DataFrame(x)
#ohe=OneHotEncoder(categorical_features=[6])
#h=ohe.fit_transform(h).toarray()
y=ds[['Bike_Buyer']]
y=pd.DataFrame(le.fit_transform(y),columns=['Bike_Buyer'])
y=y.drop([4551],axis=0)
#------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(h,y,test_size=0.1,random_state=100)
#------------------------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(random_state=0)
classifier.fit(x_train,y_train)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:60: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [28]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
wml_credentials={
    "url": "https://eu-gb.ml.cloud.ibm.com",
    "apikey": "xY05RShcE5IhIu9YgoP7xCAMXUJpK9OO7_AMOWa1GB-N",
    "username": "efd0063a-e53b-407f-934b-4643181bc7d3",
    "instance_id": "cd5063b7-4486-4586-a824-4d2256fb41f6",
    "password": "b0fe65fe-7324-4375-bca0-8ff7b311deb7",
}

In [18]:
client=WatsonMachineLearningAPIClient(wml_credentials)
model_props={client.repository.ModelMetaNames.AUTHOR_NAME:"Karthik",
             client.repository.ModelMetaNames.AUTHOR_EMAIL:"vsskarthik8@gmail.com",
             client.repository.ModelMetaNames.NAME:"Bike_Prediction"}
model_artifact=client.repository.store_model(classifier,meta_props=model_props)


published_model_uid=client.repository.get_model_uid(model_artifact)
published_model_uid
deploy=client.deployments.create(published_model_uid,name="Bike_Buyer")



#######################################################################################

Synchronous deployment creation for uid: '2631b7a9-67c7-492e-9b9a-c64838a163cd' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='623d1095-5ba8-4df4-9e4d-3e44a506fdc8'
------------------------------------------------------------------------------------------------




In [19]:
scoring_endpoint=client.deployments.get_scoring_url(deploy)

In [20]:
scoring_endpoint

'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/cd5063b7-4486-4586-a824-4d2256fb41f6/deployments/623d1095-5ba8-4df4-9e4d-3e44a506fdc8/online'